In [150]:
from sklearn.pipeline import * 
from sklearn.naive_bayes import * 
from sklearn.cluster import *  
from sklearn.covariance import *  
from sklearn.cross_decomposition import *  
from sklearn.datasets import *  
from sklearn.decomposition import *  
from sklearn.ensemble import *  
from sklearn.feature_extraction import *  
from sklearn.feature_extraction.text import *  
from sklearn.feature_selection import *  
from sklearn.gaussian_process import *  
from sklearn.linear_model import *  
from sklearn.manifold import *  
from sklearn.metrics import *  
from sklearn.mixture import *  
from sklearn.model_selection import *  
from sklearn.neighbors import *  
from sklearn.neural_network import *  
from sklearn.preprocessing import *  
from sklearn.svm import *  
from sklearn.tree import * 
from sklearn import datasets
from yellowbrick.classifier import ConfusionMatrix, ROCAUC

In [180]:
from tpot import TPOTClassifier
df = pd.read_csv('code/data/titanic_train.csv')

In [181]:
df = df.drop(['name', 'ticket', 'cabin'], axis=1)
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])

In [182]:
df = pd.get_dummies(df, columns=['embarked'])
df = df.fillna(df.mean(numeric_only=True))
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,0,3,1,22.000000,1,0,7.2500,False,False,True
1,1,1,0,38.000000,1,0,71.2833,True,False,False
2,1,3,0,26.000000,0,0,7.9250,False,False,True
3,1,1,0,35.000000,1,0,53.1000,False,False,True
4,0,3,1,35.000000,0,0,8.0500,False,False,True
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,False,False,True
887,1,1,0,19.000000,0,0,30.0000,False,False,True
888,0,3,0,29.699118,1,2,23.4500,False,False,True
889,1,1,1,26.000000,0,0,30.0000,True,False,False


In [183]:
y=df["survived"]
x = df.drop('survived', axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
cf = TPOTClassifier(generations=5, population_size=20, verbosity=2, random_state=42)

In [184]:
cf.fit(x_train, y_train)

is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_classifier


C:\Users\Yaswanth\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1230: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(
C:\Users\Yaswanth\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1270: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(


is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier


Version 0.12.2 of tpot is outdated. Version 1.0.0 was released Wednesday February 26, 2025.


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8401552795031056

Generation 2 - Current best internal CV score: 0.8401552795031056

Generation 3 - Current best internal CV score: 0.8401552795031056

Generation 4 - Current best internal CV score: 0.8401552795031056

Generation 5 - Current best internal CV score: 0.8401552795031056

Best pipeline: BernoulliNB(LogisticRegression(XGBClassifier(input_matrix, learning_rate=0.5, max_depth=3, min_child_weight=8, n_estimators=100, n_jobs=1, subsample=1.0, verbosity=0), C=0.1, dual=False, penalty=l2), alpha=10.0, fit_prior=False)


TPOTClassifier(generations=5, population_size=20, random_state=42, verbosity=2)

In [185]:
cf.score(x_test,y_test)

0.8

In [186]:
cf.predict(x_test)

array([1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0])

In [187]:
cf.export("tpot_titanic_pipe.py")

In [188]:
%pycat tpot_titanic_pipe.py

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: 0.8401552795031056
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.5, max_depth=3, min_child_weight=8, n_estimators=100, n_jobs=1, subsample=1.0, verbosity=0)),
    StackingE